In [ ]:
import time
from selenium import webdriver
from selenium.webdriver import Keys
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service

from bs4 import BeautifulSoup
import clipboard

In [ ]:
# 각 가수별로 멜론에서 검색 가능한 페이지 수를 딕셔너리로 저장
singer_count = {'남진' : 30, '김광석' : 10, '신해철': 6, '김창완':4, '주현미':24, '이문세':10, '심수봉':9, '송창식':9, '쎄시봉':9, '이선희':9, '나훈아': 72, '조영남': 24, '최진희': 14, '노사연': 6, '산울림': 11, '윤형주': 11, '해바라기': 13, '이장희': 7, '조항조': 6, '조용필': 18}

In [ ]:
# 각 가수별 노래 제목을 title_list 리스트에 각각의 리스트로 저장
title_list = []

chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option('detach',True)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),options=chrome_options)


for index, name in enumerate(singer_count): 
    title_list.append([])
    singer = name
    driver.get('https://www.melon.com/search/song/index.htm?q='+singer+'&section=&searchGnbYn=Y&kkoSpl=Y&kkoDpType=&mwkLogType=T')
    time.sleep(3)

    driver.find_element(By.XPATH, '//*[@id="conts"]/div[3]/div[1]/a[2]').click()
    time.sleep(1)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    # 1페이지.
    result = soup.find_all("a",'fc_gray')

    if len(result)>0:
        for x in result:
            title_list[-1].append(x['title'])

    for j in range(1,int(singer_count[singer]/10) + 1):
        # 2페이지부터 10페이지까지 크롤링
        for i in range(1,10):
            driver.find_element(By.XPATH, f'//*[@id="pageObjNavgation"]/div/span/a[{i}]').click()
            time.sleep(3)

            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')

            result = soup.find_all("a",'fc_gray')
            if len(result)>0:
                for x in result:
                    title_list[-1].append(x['title'])
        # 다음페이지 클릭
        driver.find_element(By.CLASS_NAME, 'btn_next').click()
        time.sleep(2)


print(title_list)

In [ ]:
singer_name = ['남진', '김광석', '신해철', '김창완', '주현미', '이문세', '심수봉', '송창식', '쎄시봉', '이선희', '나훈아', '조영남', '최진희', '노사연', '산울림', '윤형주', '해바라기', '이장희', '조항조', '조용필'}]

In [ ]:
# 노래제목과 가사를 함께 검색했을 때 가사를 반환하는 함수
import re
import requests
from bs4 import BeautifulSoup

hdr={'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
url = 'https://www.melon.com/search/lyric/index.htm?q=낙하 악뮤&section=&searchGnbYn=Y&kkoSpl=N&kkoDpType='

response = requests.get(url, headers=hdr)
val = BeautifulSoup(response.text, 'html.parser')

def get_lyrics(Q):
    hdr={'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
    url = 'https://www.melon.com/search/lyric/index.htm?q='+Q+'&section=&searchGnbYn=Y&kkoSpl=N&kkoDpType='

    response = requests.get(url, headers=hdr)
    if response.status_code == 200:
        val = response.text
        data="data-song-no="
        ID = val[val.find(data)+len(data)+1:val.find(data)+len(data)+13]

        pattern = r"\d{1,13}"
        ID = re.findall(pattern , ID)[0]
        print('Song ID : ' + ID)
        
        url2 = "https://www.melon.com/song/detail.htm?songId="+str(ID)
        response2 = requests.get(url2, headers=hdr)
        html = requests.get(url=url2, headers=hdr).text
        bs=BeautifulSoup(html, "html.parser")

        soupstr=str(bs.body.find('div', class_="lyric"))
        result=re.sub(r'<!.*?->','', soupstr)
        result=re.sub(r'<div class="lyric" id="d_video_summary">','', result)
        result=re.sub(r'</div>','', result)
        lyrics=result.replace("<br/>","\n").strip()
        
        return lyrics
    else : 
        return response.status_code

In [ ]:
import os

In [ ]:
# 노래가사를 받아오면 이를 텍스트파일로 저장
for num, singer in enumerate(singer_name):
    if not os.path.exists(singer):
        os.mkdir(singer)
    for song in title_list[num]:
        try:
            txt = get_lyrics(f'{song} {singer}')
            file = open(f'{singer}/{song}.txt','w')
            file.write(txt)
            file.close()
        except Exception:
            pass

In [ ]:
## 각 가수의 노래 가사를 가수명.txt로 하나의 텍스트로 저장
for singer in singer_name:
    for song in song_list:
        file=open(f'{singer}/{song}', 'r')
        txt=txt + '.' + file.read()
        file.close()
        file=open(f'{singer}.txt', 'w')
        file.write(txt)
        file.close()